In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import ipywidgets as widgets
from IPython.display import clear_output
from plotly.subplots import make_subplots

import plotly.graph_objects as go
import plotly.express as px
# plt.style.use('ggplot')  # Красивые графики
# plt.rcParams['figure.figsize'] = (15, 5)  # Размер картинок

In [2]:
df = pd.read_csv(r'C:\Users\koder\Downloads/forFBpost.csv',sep=";")
df=df.rename(columns={'Город':'city',
                  'Модель':'model',
                  'Нижняя граница':'lower bound',
                  'Верхняя граница':'upper bound'})
# Пробелы в имени города
df.city=df.city.str.strip()

# Для всех городов 121 запись. Есть исключения. Необходимо убрать дубликаты. Удержим первое значение
selected_rows = df[~df['fact'].isnull()]
selected_rows.city.unique()
names_cities=selected_rows.city.unique() # Все имена городов в столбце Грод

for name in names_cities:
    if (len(df[df.city==name])!=121):
        print(name, '- записей ', len(df[df.city==name]))
df=df.drop_duplicates(subset=['city','year'])

Краснослободск - записей  242
Кировск - записей  242


In [3]:
# Циклический доступ к сгруппированным фреймам для ДАШБОРДА
names_cities=df.city.unique()# Все имена городов в столбце Грод
df_grouped_cities = df.groupby('city')# Группировка по городам

# Средний цепной коэф прироста населения
# Во сколько раз за год в среденм приростает населенмие
def devide(x):
    return ((x.values[1]/x.values[0])-1)

df_coefficient=pd.DataFrame(index=names_cities)
for name in names_cities:
    df_buf=df_grouped_cities.get_group(name)
    df_coefficient.loc[name,'model_mean']=df_buf.model.rolling(window=2).apply(devide).mean()
    df_coefficient.loc[name,'fact_mean']=df_buf.fact.rolling(window=2).apply(devide).mean()

In [4]:
# Базисный коэф прироста населения 
# Во сколько раз больше/меньше стало по сравнению с началом
for name in names_cities:
    df_buf=df_grouped_cities.get_group(name)
    
    time_baza=df[['fact','year']].dropna().year.unique()[-1]
    time_finish=df_buf.year.max()

    P_baza=df_buf.loc[df_buf[df_buf.year==time_baza].index, 'model']
    P_curent=df_buf.loc[df_buf[df_buf.year==time_finish].index, 'model']
    
    TP=int(P_curent)/int(P_baza)-1
    
    df_coefficient.loc[name,'model']=TP
    
for name in names_cities:
    df_buf=df_grouped_cities.get_group(name)
    df_buf=df_buf.dropna()
    
    time_baza=df_buf.year.min()
    time_finish=df_buf.year.max()

    P_baza=df_buf.loc[df_buf[df_buf.year==time_baza].index, 'fact']
    P_curent=df_buf.loc[df_buf[df_buf.year==time_finish].index, 'fact']
    
    TP=int(P_curent)/int(P_baza)-1
    
    df_coefficient.loc[name,'fact']=TP

In [5]:
df_coefficient=df_coefficient*100
df_coefficient=df_coefficient.reset_index()
df_coefficient=df_coefficient.rename(columns={'index':'city'})

In [6]:
dfs_top5=[]
for i in ['fact','model','fact_mean','model_mean']:
    df_buf=df_coefficient.sort_values(i,ascending=False)
    df_buf=df_buf[df_buf.model_mean!=float("inf")] # бесконечная форма функции
    df_buf=df_buf[['city',i]].head(5)
    df_buf[i]=df_buf[i].astype(str) + '%'
    dfs_top5.append(df_buf)

In [7]:
from IPython.display import HTML, display

def display_table(data):
    html = "<table>"
    for row in data:
        html += "<tr>"
        for field in row:
            html += "<td><h4>%s</h4></td>"%(field)
        html += "</tr>"
    html += "</table>"
    display(HTML(html))
    
data = [[' ','','','','Топ 5 ','',' ','',' '],
        [' ','','Прирост','',' ','',' ','Темп',' '],
        ['city ','fact','city','model ','city','fact_mean ','city','model_mean '],
        ['city ','fact_mean','city','model_mean ','city','model_mean ','city','model_mean '],
        ['city ','fact_mean','city','model_mean ','city','model_mean ','city','model_mean '],
        ['city ','fact_mean','city','model_mean ','city','model_mean ','city','model_mean '],
        ['city ','fact_mean','city','model_mean ','city','model_mean ','city','model_mean '],
        ['city ','fact_mean','city','model_mean ','city','model_mean ','city','model_mean ']]
    
col=0
for i in list(range(4)):
    buf_s=dfs_top5[i]
    buf_s_cties=list(buf_s.city)
    buf_s_percent=list(buf_s.iloc[:,1])
    for j in list(range(3,8)):      
        data[j][col]=buf_s_cties[j-3]
        data[j][col+1]=buf_s_percent[j-3]
    col=col+2

In [8]:
def unique_sorted_values(array):
    unique = array.unique().tolist()
    unique.sort()
    return unique

### Функция отображения виджетов
def display_widgets():
    display(widgets.HBox([widgets.VBox([Combobox,buf_dropdown_city_multi,button_reset,output,button_top5]), 
                          widgets.VBox([dropdown_year,radio_button_type_data])]))
###

### Функции повденеия виджетов
def Combobox_eventhandler(change):
    # Если ввели название города
    if change.new in unique_sorted_values(df.city):
        #Добавить это название в объект мультилист
        buf_dropdown_city_multi.value=buf_dropdown_city_multi.value+tuple([change.new])
        # Очистить ывод и отобразить виджеты занвоо
        clear_output()
        display_widgets()  
    #Вызвать функцию обработки выбранных городов
        show_bars(buf_dropdown_city_multi.value)
        show_pie_and_plot(buf_dropdown_city_multi.value,dropdown_year.value,radio_button_type_data.value)
                
output = widgets.Output()
def button_reset_clicked(b):
    # Очистить ывод и отобразить виджеты занвоо
    clear_output()
    display_widgets()
    with output:
        buf_dropdown_city_multi.value=()

def button_top5_clicked(b):
    # Очистить ывод и отобразить виджеты занвоо
    clear_output()
    display_widgets()
    display_table(data)
        
def radio_button_type_data_eventhandler(change):
    # Очистить ывод и отобразить виджеты занвоо
    clear_output()
    display_widgets() 
    display_table(data)
    show_bars(buf_dropdown_city_multi.value)
    show_pie_and_plot(buf_dropdown_city_multi.value,dropdown_year.value,radio_button_type_data.value)
    
def dropdown_year_eventhandler(change):
    # Очистить ывод и отобразить виджеты занвоо
    clear_output()
    display_widgets()
    display_table(data)
    show_bars(buf_dropdown_city_multi.value)
    show_pie_and_plot(buf_dropdown_city_multi.value,dropdown_year.value,radio_button_type_data.value)

### Функции построения графиков на основе виджетов
    # Горизнонтальная гистограмма
def show_bars(cities):
    df_buf=df_coefficient.query('city=='+str(cities))
    
    fig = make_subplots(rows=2, cols=1,shared_xaxes=True,
                       subplot_titles=['Прирост населения',
                                       'Динамика прироста населения'])
                        
    fig.add_trace(go.Bar(y=list(df_buf.city), x=df_buf.fact.values,orientation='h',
                        base=np.zeros(len(df_buf)),
                        marker_color='#ef5438',
                        name='Фактический прирост '),
                 row=1, col=1)
    fig.add_trace(go.Bar(y=list(df_buf.city), x=df_buf.model.values,orientation='h',
                        base=np.zeros(len(df_buf)),
                        marker_color='#636efa',
                        name='Смоделированный прирост '),
                 row=1, col=1)
    
    fig.add_trace(go.Bar(y=list(df_buf.city), x=df_buf.fact_mean.values,orientation='h',
                        base=np.zeros(len(df_buf)),
                        marker_color='#ffa25a',
                        name='Фактический средний темп прироста населения'),
                 row=2, col=1)
    fig.add_trace(go.Bar(y=list(df_buf.city), x=df_buf.model_mean.values,orientation='h',
                        base=np.zeros(len(df_buf)),
                        marker_color='#12d4f3',
                        name='Смоделированный средний темп прироста населения'),
                 row=2, col=1)

    fig.update_xaxes(tickprefix="%", showgrid=True)
    fig.show()
    
    # Круговая диаграмма
def show_pie_and_plot(cities,year,type_data):
    
    fig = make_subplots(rows=1, cols=2,specs=[[{"type": "domain"},    {}]],
                       subplot_titles=['Соотношение населения. Год '+str(year),
                                       'Динамика изменения. Данные '+str(type_data)])
        
    df_buf=df.query('year=='+str(year))
    df_buf=df_buf.query('city=='+str(cities))   
    fig.add_trace(go.Pie(labels=df_buf.city,
                         values =df_buf.model),
                 row=1, col=1)
    

    df_buf=df.query('city=='+str(cities))        
    df_result=pd.DataFrame(columns=['year'],data=df_buf.year.unique())
    df_buf=df_buf.groupby('city')
    
        
    for name in cities:
#         df_result[name]=df_buf.get_group(name)[type_data].reset_index(drop=True)
        buf=df_buf.get_group(name)[[type_data,'year']].reset_index(drop=True)
        
        fig.add_trace(go.Scatter(x=buf.year, y=buf[type_data],
                        mode='lines',
                        name= name),
                     row=1, col=2)
    fig.show()    
    
###    
    

### Объявление виджетов
Combobox=widgets.Combobox(
    # value='John',
    placeholder='Choose City',
    options=unique_sorted_values(df.city),
    description='Поиск:',
    disabled=False
)

buf_dropdown_city_multi = widgets.SelectMultiple(
    options=unique_sorted_values(df.city),
    rows=10,
    description='Города',
    disabled=True)

button_reset = widgets.Button(
    description='Сбросить',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Сбросить',
    icon='uncheck' # (FontAwesome names without the `fa-` prefix)
)
button_top5 = widgets.Button(
    description='Показать Топ 5',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='"',
    icon='uncheck' # (FontAwesome names without the `fa-` prefix)
)

radio_button_type_data=widgets.RadioButtons(
    options=['model', 'fact'],
    description='Тип данных:',
    disabled=False)

dropdown_year = widgets.Dropdown(options =    unique_sorted_values(df.year),description='Год')

### Объявление их поведения
Combobox.observe(Combobox_eventhandler, names=['value'])
button_reset.on_click(button_reset_clicked)
radio_button_type_data.observe(radio_button_type_data_eventhandler, names=['value'])
dropdown_year.observe(dropdown_year_eventhandler, names='value')
button_top5.on_click(button_top5_clicked)
### Отображение
display_widgets()
